In [ ]:
import requests
import json
from datetime import datetime

# NOAA API endpoint and token
NOAA_API_TOKEN = 'mcUGYujOFyxgnHmdkedbqqfdPyAunRyQ'
BASE_URL = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

# Set your request parameters
params = {
    'datasetid': 'GHCND',             # Daily summaries dataset
    'stationid': 'GHCND:USW00023234', # Example: Los Angeles International Airport
    'startdate': '2024-03-25',
    'enddate': '2024-03-25',
    'limit': 1000,
    'units': 'standard'
}

headers = {
    'token': NOAA_API_TOKEN
}

# Fetch data from NOAA API
response = requests.get(BASE_URL, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))
else:
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
import requests
import json

# NOAA token and endpoint
API_TOKEN = 'mcUGYujOFyxgnHmdkedbqqfdPyAunRyQ'
API_URL = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

HEADERS = {'token': API_TOKEN}
PARAMS = {
    'datasetid': 'GHCND',
    'stationid': 'GHCND:USW00023234',
    'startdate': '2024-03-25',
    'enddate': '2024-03-25',
    'limit': 1000,
    'units': 'standard'
}

# Send request
response = requests.get(API_URL, headers=HEADERS, params=PARAMS)

# Process and structure data
if response.status_code == 200:
    data = response.json()['results']
    
    # Build a flat dictionary of key metrics
    record = {'date': data[0]['date'], 'station': data[0]['station']}
    for entry in data:
        record[entry['datatype']] = entry['value']
    
    print("Structured Weather Record:")
    print(json.dumps(record, indent=2))

else:
    print(f"Error: {response.status_code} - {response.text}")


In [6]:
import requests
import json
import boto3

# ========================
# NOAA API Setup
# ========================
API_TOKEN = 'mcUGYujOFyxgnHmdkedbqqfdPyAunRyQ'
API_URL = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

HEADERS = {'token': API_TOKEN}
PARAMS = {
    'datasetid': 'GHCND',
    'stationid': 'GHCND:USW00023234',
    'startdate': '2025-04-02',
    'enddate': '2025-04-02',
    'limit': 1000,
    'units': 'standard'
}

# ========================
# AWS Kinesis Setup
# ========================
REGION_NAME = 'us-east-1'  # Replace with your AWS region
STREAM_NAME = 'weather-stream'  # Replace with your actual Kinesis stream name

kinesis = boto3.client('kinesis', region_name=REGION_NAME)

# ========================
# Fetch and Send Weather Data
# ========================
response = requests.get(API_URL, headers=HEADERS, params=PARAMS)

if response.status_code == 200:
    data = response.json()['results']

    # Flatten the data
    record = {'date': data[0]['date'], 'station': data[0]['station']}
    for entry in data:
        record[entry['datatype']] = entry['value']

    print("Structured Weather Record:")
    print(json.dumps(record, indent=2))

    # Send to Kinesis
    try:
        response = kinesis.put_record(
            StreamName= 'weather-stream',
            Data=json.dumps(record),
            PartitionKey="weather-data"
        )
        print("✅ Successfully sent to Kinesis:")
        print(response)
    except Exception as e:
        print("❌ Error sending to Kinesis:", str(e))

else:
    print(f"Error: {response.status_code} - {response.text}")


Structured Weather Record:
{
  "date": "2025-04-02T00:00:00",
  "station": "GHCND:USW00023234",
  "AWND": 14.5,
  "PRCP": 0.0,
  "TAVG": 51.0,
  "TMAX": 59.0,
  "TMIN": 44.0,
  "WDF2": 270,
  "WDF5": 280,
  "WSF2": 31.1,
  "WSF5": 38.9,
  "WT08": 1
}
✅ Successfully sent to Kinesis:
{'ShardId': 'shardId-000000000001', 'SequenceNumber': '49662841736262626213428755319003886154041311383808638994', 'ResponseMetadata': {'RequestId': 'f9be5133-b29e-3f45-9935-2913233e7218', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f9be5133-b29e-3f45-9935-2913233e7218', 'x-amz-id-2': 'M88jOMPw0Lq4sXV0IuDiyO+mN7K3UlqnHxe6/Z4o+cMfpcVwdBv2TUHvSMciCaRNOdexvvIL+ZEDhyKSYKAR93o2nIW4UktG', 'date': 'Wed, 30 Apr 2025 00:59:12 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [4]:
import requests
import json
import boto3

# ========================
# NOAA API Setup
# ========================
API_TOKEN = 'mcUGYujOFyxgnHmdkedbqqfdPyAunRyQ'
API_URL = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

HEADERS = {'token': API_TOKEN}
PARAMS = {
    'datasetid': 'GHCND',
    'stationid': 'GHCND:USW00023234',
    'startdate': '2025-04-01',
    'enddate': '2025-04-01',
    'limit': 1000,
    'units': 'standard'
}

# ========================
# AWS Kinesis Setup
# ========================
REGION_NAME = 'us-east-1'
STREAM_NAME = 'weather-stream'

kinesis = boto3.client('kinesis', region_name=REGION_NAME)

# ========================
# Fetch and Send Weather Data
# ========================
response = requests.get(API_URL, headers=HEADERS, params=PARAMS)

if response.status_code == 200:
    response_json = response.json()
    
    if 'results' in response_json:
        data = response_json['results']

        # Flatten the data
        record = {'date': data[0]['date'], 'station': data[0]['station']}
        for entry in data:
            record[entry['datatype']] = entry['value']

        print("Structured Weather Record:")
        print(json.dumps(record, indent=2))

        # Send to Kinesis
        try:
            kinesis_response = kinesis.put_record(
                StreamName=STREAM_NAME,
                Data=json.dumps(record),
                PartitionKey="weather-data"
            )
            print("✅ Successfully sent to Kinesis:")
            print(kinesis_response)
        except Exception as e:
            print("❌ Error sending to Kinesis:", str(e))
    else:
        print("⚠️ 'results' key not found in API response.")
        print("Full response:")
        print(json.dumps(response_json, indent=2))
else:
    print(f"❌ API Error: {response.status_code}")
    print(response.text)


Structured Weather Record:
{
  "date": "2025-04-01T00:00:00",
  "station": "GHCND:USW00023234",
  "AWND": 15.9,
  "PRCP": 0.32,
  "TAVG": 52.0,
  "TMAX": 57.0,
  "TMIN": 49.0,
  "WDF2": 280,
  "WDF5": 280,
  "WSF2": 30.0,
  "WSF5": 40.0,
  "WT01": 1,
  "WT08": 1
}
✅ Successfully sent to Kinesis:
{'ShardId': 'shardId-000000000001', 'SequenceNumber': '49662841736262626213428755251955651272394345957877088274', 'ResponseMetadata': {'RequestId': 'cf2add1f-21f3-1019-afa1-aa332226d316', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cf2add1f-21f3-1019-afa1-aa332226d316', 'x-amz-id-2': 'wU5v36hJgaN+XJcQt19wVRNEJOqoKku7qcs34asv5Mpw3kFML1oWJeYITEprFyLJxWI3jbm30YTH7oXFcZ3U3AkoGt2NMx4rj5uoPIZjzF0=', 'date': 'Wed, 30 Apr 2025 00:55:09 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
